In [ ]:
instructions = open('input1.txt', 'r').read()
floor = 0
hit_basement = False
first_basement = 0
for index, instruction in enumerate(instructions):
    if instruction == '(':
        floor += 1
    else:
        floor -= 1
    if floor < 0 and not hit_basement:
        first_basement = index+1
        hit_basement = True
print(f'Santa ends up on floor {floor}')
print(f'He first hits the basement on instruction {first_basement}')

In [ ]:
boxes = [[int(x) for x in line.split('x')] for line in open('input2.txt', 'r').readlines()]
paper_area = 2 * sum(x*y + x*z + y*z + 0.5*min(x*y, x*z, y*z) for x,y,z in boxes)
print(f'The elves need {paper_area} square feet of wrapping paper')

ribbon = sum(min(2*(x+y), 2*(x+z), 2*(y+z)) + x*y*z for x,y,z in boxes)
print(f'The elves need {ribbon} feet of ribbon')

In [ ]:
instructions = open('input3.txt', 'r').read()
visited = set()
doubles = set()

first_santa = (0,0)
normal_santa = (0,0)
robo_santa = (0,0)

first_visited = set()
second_visited = set()

first_visited.add(first_santa)
second_visited.add(robo_santa)

def move(coordinate, instruction):
    x, y = coordinate
    if instruction == '>':
        return (x+1, y)
    elif instruction == '<':
        return (x-1, y)
    elif instruction == 'v':
        return (x, y-1)
    elif instruction == '^':
        return (x, y+1)

for index, instruction in enumerate(instructions):
    first_santa = move(first_santa, instruction)
    first_visited.add(first_santa)
    
    if index % 2 == 0:
        normal_santa = move(normal_santa, instruction)
        second_visited.add(normal_santa)
    else:
        robo_santa = move(robo_santa, instruction)
        second_visited.add(robo_santa)

print(f'Santa has visited {len(first_visited)} houses')
print(f'Santa and Robo-Santa have visited {len(second_visited)} houses')
    

In [ ]:
import hashlib

input = 'yzbqklnj'
counter = 0

five_digits = 1e1000
six_digits = 1e1000

while True:
    m = hashlib.md5()
    m.update(f'{input}{counter}'.encode('utf-8'))
    if m.hexdigest().startswith('00000'):
        five_digits = counter if counter < five_digits else five_digits
    if m.hexdigest().startswith('000000'):
        six_digits = counter if counter < six_digits else six_digits
        break
    counter += 1
print(f'The lowest integer to produce a hash starting with five zeros is {five_digits}')
print(f'The lowest integer to produce a hash starting with six zeros is {six_digits}')

In [ ]:
import re
strings = open('input5.txt', 'r').readlines()

forbidden = '(ab|cd|pq|xy)'
double = '(.)\\1'
vowel = '[aeiou].*[aeiou].*[aeiou]'

def is_nice(string):
    return re.search(forbidden, string) is None and re.search(double, string) is not None and re.search(vowel, string) is not None

print(f'There are {sum(1 if is_nice(string) else 0 for string in strings)} nice strings')

double_repetition = '(..).*\\1'
space_repetition = '(.).\\1'

def is_new_nice(string):
    return re.search(double_repetition, string) is not None and re.search(space_repetition, string) is not None

print(f'There are {sum(1 if is_new_nice(string) else 0 for string in strings)} nice strings by the new rules')

In [ ]:
import re
from shapely.geometry import Polygon, Point
from shapely import intersects, intersection, unary_union

instructions = open('input6.txt', 'r').readlines()
reader = '(turn on|turn off|toggle) ([0-9]+),([0-9]+) through ([0-9]+),([0-9]+)'
turned_on = None
strengths = {}

for index, instruction in enumerate(instructions):
    print(f'\rPerforming instruction {index+1} of {len(instructions)}', end='')
    match = re.search(reader, instruction)
    x1 = int(match.group(2))
    x2 = int(match.group(4))
    y1 = int(match.group(3))
    y2 = int(match.group(5))

    area = Polygon([(x1, y1), (x1, y2+1), (x2+1, y2+1), (x2+1, y1)])
    if intersects(area, turned_on):
        affected = area.difference(turned_on)
        intersected = intersection(area, turned_on)
        still_on = turned_on.difference(area)
    
        if match.group(1) == 'turn on':
            turned_on = unary_union([affected, intersected, still_on])
        elif match.group(1) == 'toggle':
            turned_on = unary_union([affected, still_on])
        elif match.group(1) == 'turn off':
            turned_on = still_on
    else:
        if match.group(1) == 'turn on' or match.group(1) == 'toggle':
            turned_on = unary_union([area, turned_on])

    new_strengths = {}
    for strength_area in strengths:
        strength = strengths[strength_area]

        if intersects(area, strength_area):
            affected = intersection(area, strength_area)
            unaffected = strength_area.difference(area)
            area = area.difference(strength_area)

            new_strengths[unaffected] = strength
            if match.group(1) == 'turn on':
                new_strengths[affected] = strength + 1
            elif match.group(1) == 'toggle':
                new_strengths[affected] = strength + 2
            elif match.group(1) == 'turn off' and strength > 1:
                new_strengths[affected] = strength - 1
        else:
            new_strengths[strength_area] = strength

    if area.area > 0:
        if match.group(1) == 'turn on':
            new_strengths[area] = 1
        elif match.group(1) == 'toggle':
            new_strengths[area] = 2

    strengths = {unary_union([area for area, area_strength in new_strengths.items() if area_strength == strength]): strength for strength in set(new_strengths.values())}
        

print(f'\nThere are {turned_on.area} lights turned on')
print(f'The total brightness is {sum(area.area * strength for area, strength in strengths.items())}')

In [ ]:
import re
instructions = open('input7.txt', 'r').readlines()

gates = {}

single_input = '^(\w+) -> (\w+)$'
not_input = '^NOT (\w+) -> (\w+)$'
and_input = '^(\w+) AND (\w+) -> (\w+)$'
or_input = '^(\w+) OR (\w+) -> (\w+)$'
lshift_input = '^(\w+) LSHIFT (\w+) -> (\w+)$'
rshift_input = '^(\w+) RSHIFT (\w+) -> (\w+)$'

target_check = '-> (\w+)$'
non_overwritable_target = ''

def get_value(input):
    if input.isdigit():
        return int(input)
    elif input in gates:
        return gates[input]

    return None

def perform_instructions():
    while len(instructions) > 0 and 'a' not in gates:
        performed_indices = []
        for index, instruction in enumerate(instructions):
            if re.search(target_check, instruction).group(1) == non_overwritable_target:
                print(instruction)
                performed_indices.append(index)
                continue
            
            performed = False
            if (match := re.search(single_input, instruction)) is not None:
                if (value := get_value(match.group(1))) is not None:
                    gates[match.group(2)] = value
                    performed = True
                    
            elif (match := re.search(not_input, instruction)) is not None:
                if (value := get_value(match.group(1))) is not None:
                    gates[match.group(2)] = value ^ 65535
                    performed = True
    
            elif (match := re.search(and_input, instruction)) is not None:
                if (value1 := get_value(match.group(1))) is not None and (value2 := get_value(match.group(2))) is not None:
                    gates[match.group(3)] = value1 & value2
                    performed = True
    
            elif (match := re.search(or_input, instruction)) is not None:
                if (value1 := get_value(match.group(1))) is not None and (value2 := get_value(match.group(2))) is not None:
                    gates[match.group(3)] = value1 | value2
                    performed = True
    
            elif (match := re.search(lshift_input, instruction)) is not None:
                if (value1 := get_value(match.group(1))) is not None and (value2 := get_value(match.group(2))) is not None:
                    gates[match.group(3)] = (value1 << value2) & 65535
                    performed = True
    
            elif (match := re.search(rshift_input, instruction)) is not None:
                if (value1 := get_value(match.group(1))) is not None and (value2 := get_value(match.group(2))) is not None:
                    gates[match.group(3)] = (value1 >> value2) & 65535
                    performed = True
    
            if performed:
                performed_indices.append(index)
    
        for performed_index in sorted(performed_indices, reverse=True):
            del instructions[performed_index]

perform_instructions()
print(f'The signal on wire a is {gates["a"]}')

gates = {'b': gates['a']}
non_overwritable_target = 'b'
instructions = open('input7.txt', 'r').readlines()
perform_instructions()
print(f'The new signal on wire a is {gates["a"]}')

In [ ]:
input = open('input8.txt', 'r').readlines()
bytes = sum(len(line) for line in input)
characters = bytes
longer = bytes
for line in input:
    characters -= 2
    i = 0
    while i < len(line):
        if line[i:].startswith('\\\\'):
            characters -= 1
            i += 2
        elif line[i:].startswith('\\"'):
            characters -= 1
            i += 2
        elif line[i:].startswith('\\x'):
            characters -= 3
            i += 4
        else:
            i += 1
print(f'The answer to part one is {bytes - characters}')

for line in input:
    longer += 2
    for c in line:
        if c == '\\':
            longer += 1
        elif c == '"':
            longer += 1

print(f'The answer to part two is {longer - bytes}')

In [ ]:
import re, itertools
locations = {}

for line in open('input9.txt', 'r').readlines():
    if (match := re.search('(\w+) to (\w+) = ([0-9]+)', line)):
        place_one = match.group(1)
        place_two = match.group(2)
        distance = int(match.group(3))

        if place_one not in locations:
            locations[place_one] = {}
        if place_two not in locations:
            locations[place_two] = {}

        locations[place_one][place_two] = distance
        locations[place_two][place_one] = distance

def find_distance(locations_list):
    distance = 0
    for i in range(1, len(locations_list)):
        distance += locations[locations_list[i-1]][locations_list[i]]
    return distance

shortest_path = min(find_distance(route) for route in itertools.permutations(locations.keys()))
longest_path = max(find_distance(route) for route in itertools.permutations(locations.keys()))
print(f'The shortest possible path is {shortest_path}')
print(f'The longest possible path is {longest_path}')

In [ ]:
from itertools import groupby

start = '3113322113'

def look_and_say(input_string, num_iterations):
    for i in range(num_iterations):
        input_string = ''.join([str(len(list(g))) + str(k) for k, g in groupby(input_string)])
    return input_string

built_string = look_and_say(start, 40)
print(f'The length of the string after 40 times is {len(built_string)}')
built_string_longer = look_and_say(start, 50)
print(f'The length of the string after 50 times is {len(built_string_longer)}')

In [ ]:
password = 'hepxcrrq'

def increment(password_string):
    last = ord(password_string[-1])
    if last == 122:
        return increment(password_string[:-1]) + 'a'
    else:
        return password_string[:-1] + chr(last+1)

def check_rules(password):
    if 'i' in password or 'o' in password or 'l' in password:
        return False

    pairs = 0
    has_straight = False
    counter = 0
    while counter < len(password):
        try:
            if password[counter] == password[counter + 1]:
                pairs += 1
                counter += 1
    
                if ord(password[counter+1]) - ord(password[counter]) == 1 and ord(password[counter+2]) - ord(password[counter+1]) == 1:
                    has_straight = True
        except IndexError:
            pass

        counter += 1

    return pairs > 1 and has_straight

incrementing_password = password
while not check_rules(incrementing_password):
    incrementing_password = increment(incrementing_password)
print(f'The next password that follows all the rules is {incrementing_password}')
incrementing_password = increment(incrementing_password)
while not check_rules(incrementing_password):
    incrementing_password = increment(incrementing_password)
print(f'The next password is {incrementing_password}')

In [ ]:
import re, json
input = open('input12.txt', 'r').read()
matches = re.findall('[-]{0,1}[0-9]+', input)
print(f'The sum of all numbers contained in the document is {sum(int(x) for x in matches)}')

# Remove all objects with red as a property
python = json.loads(input)

def remove_red(object):
    if isinstance(object, list):
        return [remove_red(child) for child in object if remove_red(child) is not None]
    elif isinstance(object, dict):
        new_object = {}
        for key in object:
            if object[key] == 'red':
                return None
            new_object[key] = remove_red(object[key])
        return new_object
    else:
        return object


new_python = remove_red(python)
redless = json.dumps(new_python)
print(f'The sum of all numbers without red is {sum(int(x) for x in re.findall("[-]{0,1}[0-9]+", redless))}')

In [ ]:
import re
from itertools import permutations
input = open('input13.txt', 'r').readlines()

people = set()
happiness = {}

for line in input:
    match = re.match('(.+) would (gain|lose) ([0-9]+) happiness units by sitting next to (.+)\.', line)
    person_one = match.group(1)
    person_two = match.group(4)
    gain = int(match.group(3)) * (1 if match.group(2) == 'gain' else -1)

    people.add(person_one)
    if person_one not in happiness:
        happiness[person_one] = {}
    happiness[person_one][person_two] = gain

def calculate_happiness(list):
    sum = 0
    for i in range(len(list)):
        sum += happiness[list[i]][list[(i+1) % len(list)]] + happiness[list[(i+1) % len(list)]][list[i]]
    return sum

print(f'The maximal change in happiness is {max(calculate_happiness(arrangement) for arrangement in permutations(people))}')

happiness['Me'] = {}
for person in people:
    happiness['Me'][person] = 0
    happiness[person]['Me'] = 0
people.add('Me')

print(f'The maximal change in happiness including me is {max(calculate_happiness(arrangement) for arrangement in permutations(people))}')

In [ ]:
import re
reindeer = []
input = open('input14.txt', 'r').readlines()

for line in input:
    match = re.match('(.+) can fly ([0-9]+) km/s for ([0-9]+) seconds, but then must rest for ([0-9]+) seconds.', line)
    deer = {
        'name': match.group(1),
        'speed': int(match.group(2)),
        'flying_time': int(match.group(3)),
        'resting_time': int(match.group(4))
    }
    reindeer.append(deer)

def calculate_distance(time, reindeer):
    distances = []
    for deer in reindeer:
        period = deer['flying_time'] + deer['resting_time']
        distance_per_period = deer['speed'] * deer['flying_time']
        total_distance = distance_per_period * (time // period) + min(deer['flying_time'], (time % period)) * deer['speed']
        distances.append(total_distance)
    return distances

print(f'After 2503 seconds, the winning reindeer has travelled {max(calculate_distance(2503, reindeer))} km')

scores = [0 for deer in reindeer]
for i in range(1, 2504):
    distances = calculate_distance(i, reindeer)
    farthest = max(distances)
    for i in range(len(distances)):
        if distances[i] == farthest:
            scores[i] += 1
print(f'The most-scoring reindeer has {max(scores)} points')

In [ ]:
import re
from itertools import permutations

input = open('input15.txt', 'r').readlines()
ingredients = []
for line in input:
    match = re.match('(.+): capacity ([-0-9]+), durability ([-0-9]+), flavor ([-0-9]+), texture ([-0-9]+), calories ([-0-9]+)', line)
    ingredient = {
        'capacity': int(match.group(2)),
        'durability': int(match.group(3)),
        'flavor': int(match.group(4)),
        'texture': int(match.group(5)),
        'calories': int(match.group(6))
    }
    ingredients.append(ingredient)

score_types = ['capacity', 'durability', 'flavor', 'texture']
distributions = set()
for a in range(101):
    for b in range(101 - a):
        for c in range(101 - a - b):
            d = 100 - a - b -c
            if d < 0:
                break
            distributions.add(tuple(sorted([a, b, c, d])))

distributions = list(distributions)
recipes = []
low_cal_recipes = []
for distribution in distributions:
    for permutation in set(permutations(distribution)):
        score = 1
        for score_type in score_types:
            type_score = sum(permutation[i] * ingredients[i][score_type] for i in range(len(ingredients)))
            if type_score < 0:
                type_score = 0
            score *= type_score
        recipes.append((score, *distribution))
        calories = sum(permutation[i] * ingredients[i]['calories'] for i in range(len(ingredients)))
        if calories == 500:
            low_cal_recipes.append((score, *distribution))
print(f'The maximum recipe score is {max(recipe[0] for recipe in recipes)}')
print(f'The maximum 500cal cookie is {max(recipe[0] for recipe in low_cal_recipes)}')

In [ ]:
import re

input = open('input16.txt', 'r').readlines()
detectables = ['children', 'cats', 'samoyeds', 'pomeranians', 'akitas', 'vizslas','goldfish', 'trees', 'cars', 'perfumes']
sue_facts = [
    {
        match.group(1): int(match.group(2)) for detectable in detectables for match in [re.search(f'({detectable}): ([0-9]+)', line)] if match is not None
    }
    for line in input
]

requirements = {
    'children': 3,
    'cats': 7,
    'samoyeds': 2,
    'pomeranians': 3,
    'akitas': 0,
    'vizslas': 0,
    'goldfish': 5,
    'trees': 3,
    'cars': 2,
    'perfumes': 1
}

def check_sue(sue):
    for detectable in sue:
        if sue[detectable] != requirements[detectable]:
            return False
    return True

def check_sue_modified(sue):
    for detectable in sue:
        if detectable == 'cats' or detectable == 'trees' or detectable == 'pomeranians' or detectable == 'goldfish':
            if (detectable == 'cats' or detectable == 'trees') and sue[detectable] <= requirements[detectable]:
                return False
            elif (detectable == 'pomeranians' or detectable == 'goldfish') and sue[detectable] >= requirements[detectable]:
                return False
        elif sue[detectable] != requirements[detectable]:
            return False
    return True

matching_sues = [index+1 for index, sue in enumerate(sue_facts) if check_sue(sue)]
modified_sues = [index+1 for index, sue in enumerate(sue_facts) if check_sue_modified(sue)]
print(f'The aunt Sue that bought the gift is number {matching_sues[0]}')
print(f'The corrected aunt Sue that bought the gift is number {modified_sues[0]}')

In [ ]:
capacities = sorted([int(x.strip()) for x in open('input17.txt', 'r').readlines()])
totals = {}
for i in range(2**len(capacities)):
    total_capacity = 0
    containers = []
    for j, capacity in enumerate(capacities):
        if (0b1 << j) & i:
            total_capacity += capacity
            containers.append(j)
    totals[tuple(containers)] = total_capacity
print(f'The number of combinations that can fit 150 liters of eggnog is {sum(1 for containers in totals if totals[containers] == 150)}')
minimum_containers = min(len(containers) for containers in totals if totals[containers] == 150)
print(f'The number of combinations that use only {minimum_containers} containers is {sum(1 for containers in totals if totals[containers] == 150 and len(containers) == minimum_containers)}')

In [ ]:
input = open('input18.txt', 'r').readlines()
lights = {y*100 + x: {'on': input[y][x] == '#', 'neighbours': []} for y in range(100) for x in range(100)}
stuck_lights = {y*100 + x: {'on': input[y][x] == '#', 'neighbours': []} for y in range(100) for x in range(100)}
for x in range(100):
    for y in range(100):
        light = lights[y*100 + x]
        stuck_light = stuck_lights[y*100 + x]
        for j in [-1, 0, 1]:
            for i in [-1, 0, 1]:
                if i == 0 and j == 0:
                    continue
                if x+i >= 100 or x+i < 0:
                    continue
                if y+j >= 100 or y+j < 0:
                    continue
                possible_neighbour = (100*(y+j) + x+i)
                if possible_neighbour in lights:
                    light['neighbours'].append(lights[possible_neighbour])
                    stuck_light['neighbours'].append(stuck_lights[possible_neighbour])

def update_lights(lights):
    new_states = {}
    for light in lights:
        neighbours_on = sum(1 if neighbour['on'] else 0 for neighbour in lights[light]['neighbours'])
        if lights[light]['on'] and (neighbours_on == 2 or neighbours_on == 3):
            new_states[light] = True
        elif lights[light]['on']:
            new_states[light] = False
        elif neighbours_on == 3:
            new_states[light] = True
        else:
            new_states[light] = False

    for light in lights:
        lights[light]['on'] = new_states[light]

def update_stuck_lights(stuck_lights):
    stuck_lights[0]['on'] = True
    stuck_lights[99]['on'] = True
    stuck_lights[9900]['on'] = True
    stuck_lights[9999]['on'] = True
    update_lights(stuck_lights)  
    stuck_lights[0]['on'] = True
    stuck_lights[99]['on'] = True
    stuck_lights[9900]['on'] = True
    stuck_lights[9999]['on'] = True
    

for i in range(100):
    update_lights(lights)
    update_stuck_lights(stuck_lights)

print(f'After 100 steps, there are {sum(1 if light["on"] else 0 for light in lights.values())} lights on')
print(f'After 100 steps, there are {sum(1 if light["on"] else 0 for light in stuck_lights.values())} lights on in the stuck grid')

In [ ]:
import re
transitions, molecule = open('input19.txt', 'r').read().split('\n\n')
molecule = molecule.strip()

transition_rules = []
for transition in transitions.split('\n'):
    match = re.match('(.+) => (.+)', transition)

    transition_rules.append((match.group(1), match.group(2)))

def apply_transition(molecule, transition_rules):
    new_molecules = set()
    for transition_rule, target in transition_rules:
        for i in range(len(molecule)):
            if molecule[i:].startswith(transition_rule):
                new_molecule = molecule[:i] + target + molecule[i+len(transition_rule):]
                new_molecules.add(new_molecule)

    return new_molecules

print(f'The number of new molecules creatable is {len(apply_transition(molecule, transition_rules))}')

reverse_transition_rules = [(target, source) for source, target in transition_rules]

def depth_search(string, target, transition_rules, level):
    new_molecules = apply_transition(string, reverse_transition_rules)
    new_molecules = sorted(list(new_molecules), key=lambda m:len(m))
    for new_molecule in new_molecules:
        if new_molecule == target:
            return level

        depth = depth_search(new_molecule, target, transition_rules, level+1)
        if depth is not None:
            return depth
    return None

steps_required = depth_search(molecule, 'e', reverse_transition_rules, 1)
print(f'The number of steps required to make the medicine is {steps_required}')

In [ ]:
import math
present_threshold = 36000000


def calculate_lowest_house_number(present_threshold):
    sum_of_factors_threshold = present_threshold / 10
    lowest_house_number = 2**(math.ceil(math.log(1 + sum_of_factors_threshold*2 - sum_of_factors_threshold)/math.log(2) - 1))
    lowest_new_house_number = lowest_house_number

    primes = []
    non_primes = set()
    max_prime = math.floor(math.sqrt(lowest_house_number))
    for i in range(2, max_prime):
        if i in non_primes:
            continue
        primes.append(i)
        for factor in range(2, (max_prime // i) + 1):
            non_primes.add(i*factor)

    def calculate_sum_of_factors(prime_nos):
        sum = 1
        for index, prime_no in enumerate(prime_nos):
            sum *= (primes[index]**(prime_no + 1) - 1) / (primes[index] - 1)
        return sum

    def calculate_sum_of_factors_new_rules(prime_nos):
        number = 1
        for prime_index, prime in enumerate(primes):
            number *= prime**prime_nos[prime_index]

        factors = []
        current_primes = [0 for prime in primes]
        while current_primes[0] <= prime_nos[0]:
            factor = 1
            for prime_index, prime in enumerate(primes):
                factor *= prime**current_primes[prime_index]
            if factor >= number / 50:
                factors.append(factor)

            for index in range(len(current_primes)-1, -1, -1):
                if current_primes[index] < prime_nos[index] or index == 0:
                    current_primes[index] += 1
                    break
                else:
                    current_primes[index] = 0
        return sum(factors)

    for i in range(900000, 2, -1):
        number = i
        factors = [0 for i in range(len(primes))]
        for prime_index, prime in enumerate(primes):
            while number % prime == 0:
                number = number // prime
                factors[prime_index] += 1

        sum_of_factors = calculate_sum_of_factors(factors)
        sum_of_factors_new_rules = calculate_sum_of_factors_new_rules(factors)
        if sum_of_factors >= sum_of_factors_threshold:
            lowest_house_number = i
        if sum_of_factors_new_rules * 11 >= present_threshold:
            lowest_new_house_number = i
        if i % 10000 == 0:
            print(f'\rInvestigating house {i}. Current lowest house number is {lowest_house_number}. Current new rules lowest {lowest_new_house_number}                 ', end='')
    print(f'\nLowest house number is {lowest_house_number}')
    print(f'Lowest new rules house number is {lowest_new_house_number}')
        

calculate_lowest_house_number(present_threshold)

In [ ]:
weapons = [
    (8, 4, 0),
    (10, 5, 0),
    (25, 6, 0),
    (40, 7, 0),
    (74, 7, 0)
]

armor = [
    (0, 0, 0),
    (13, 0, 1),
    (31, 0, 2),
    (53, 0, 3),
    (75, 0, 4),
    (102, 0, 5)
]

rings = [
    (0, 0, 0),
    (25, 1, 0),
    (50, 2, 0),
    (100, 3, 0),
    (20, 0, 1),
    (40, 0, 2),
    (80, 0, 3)
]

setups = []
for weapon_price, weapon_damage, weapon_armor in weapons:
        for armor_price, armor_damage, armor_armor in armor:
            setups.append((weapon_price + armor_price, weapon_damage + armor_damage, weapon_armor + armor_armor))
            for i in range(len(rings)):
                for j in range(i+1, len(rings)):
                    ring_one_price, ring_one_damage, ring_one_armor = rings[i]
                    ring_two_price, ring_two_damage, ring_two_armor = rings[j]
                    setups.append((weapon_price + armor_price + ring_one_price + ring_two_price, weapon_damage + armor_damage + ring_one_damage + ring_two_damage, weapon_armor + armor_armor + ring_one_armor + ring_two_armor))

setups.sort(key=lambda s: s[0])

boss_hit_points = 103
boss_damage = 9
boss_armor = 2

def fight(setup):
    bh = boss_hit_points
    ph = 100

    bd = boss_damage
    pd = setup[1]

    ba = boss_armor
    pa = setup[2]

    while bh > 0 and ph > 0:
        bh -= max((pd - ba), 1)
        if bh > 0:
            ph -= max((bd - pa), 1)

    if ph > 0:
        return True
    else:
        return False

for setup in setups:
    if fight(setup):
        print(f'The player can win by buying {setup[0]} coins of equipment')
        break

for setup in setups[::-1]:
    if not fight(setup):
        print(f'The player can lose while buying {setup[0]} coins of equipment')
        break

In [ ]:
spells = [
    (53, 4, 0, 0, 0, 0),
    (73, 2, 2, 0, 0, 0),
    (113, 0, 0, 7, 0, 6),
    (173, 3, 0, 0, 0, 6),
    (229, 0, 0, 0, 101, 5)
]

ph = 50
pm = 500
bh = 58
bd = 9

#ph = 10
#pm = 250
#bh = 14
#bd = 8

minimum = 1e100

def play_game(player_health, player_mana, boss_health, boss_damage, chosen_spell, old_effects, spent_mana):
    global minimum
    # Copy effects, since we're doing this recursively
    effects = {effect: old_effects[effect] for effect in old_effects}

    ph = player_health
    pm = player_mana
    bh = boss_health
    bd = boss_damage
    sm = spent_mana

    ph -= 1
    if ph <= 0:
        return None

    # Player turn, go through all effects
    def go_through_effects(ph, pm, bh, bd):
        pa = 0
        to_delete = []
        for effect in effects:
            _, damage, healing, armor, recharge, _ = spells[effect]
            remaining_duration = effects[effect]
    
            bh -= damage
            ph += healing
            pa += armor
            pm += recharge
    
            if remaining_duration == 1:
                to_delete.append(effect)
            else:
                effects[effect] -= 1

        for effect in to_delete:
            del effects[effect]
        return ph, pm, bh, bd, pa

    ph, pm, bh, bd, pa = go_through_effects(ph, pm, bh, bd)
    if bh <= 0:
        if sm < minimum:
            minimum = sm
        return sm

    # Use chosen spell
    mana_cost, damage, healing, armor, recharge, duration = spells[chosen_spell]
    pm -= mana_cost
    sm += mana_cost

    if sm > minimum:
        return None

    if duration != 0:
        effects[chosen_spell] = duration
    else:
        bh -= damage
        ph += healing
        pa += armor
        pm += recharge

    if bh <= 0:
        if sm < minimum:
            minimum = sm
        return sm

    # Boss turn
    ph, pm, bh, bd, pa = go_through_effects(ph, pm, bh, bd)

    if bh <= 0:
        if sm < minimum:
            minimum = sm
        return sm

    # Boss attack
    ph -= max(bd - pa, 1)

    if ph <= 0:
        return None

    # Find valid spells to use next
    # Special case, player is being given mana
    mana_next_round = pm + sum(spells[effect][4] for effect in effects)
    spells_to_use = []
    for spell_index, (mana_cost, _, _, _, _, _) in enumerate(spells):
        if mana_cost > mana_next_round:
            continue
        if spell_index in effects and effects[spell_index] > 1:
            continue
        spells_to_use.append(spell_index)
    if len(spells_to_use) == 0:
        return None

    continuations = [play_game(ph, pm, bh, bd, spell, effects, sm) for spell in spells_to_use]
    if all(continuation is None for continuation in continuations):
        return None
    return min(cost for cost in continuations if cost is not None)

# At the beginning, all spells are valid
valid_spells = [i for i in range(len(spells))]
costs = [play_game(ph, pm, bh, bd, spell, {}, 0) for spell in valid_spells]
print(costs)
print(f'The least amount of mana spent to beat the boss is {min(cost for cost in costs if cost is not None)}')

In [ ]:
import re

input = open('input23.txt', 'r').readlines()

def read_instruction(line_number, a, b):
    line = input[line_number]
    has_jumped = False
    half_match = re.search('hlf (a|b)', line)
    if half_match is not None:
        if half_match.group(1) == 'a':
            a //= 2
        else:
            b //= 2

    triple_match = re.search('tpl (a|b)', line)
    if triple_match is not None:
        if triple_match.group(1) == 'a':
            a *= 3
        else:
            b *= 3

    increment_match = re.search('inc (a|b)', line)
    if increment_match is not None:
        if increment_match.group(1) == 'a':
            a += 1
        else:
            b += 1

    jump_match = re.search('jmp [\\+]{0,1}([-0-9]+)', line)
    if jump_match is not None:
        has_jumped = True
        line_number += int(jump_match.group(1))

    cond_jump_match = re.search('(jio|jie) (a|b), [\\+]{0,1}([-0-9]+)', line)
    if cond_jump_match is not None:
        register_value = a if cond_jump_match.group(2) == 'a' else b
        should_jump = register_value == 1 if cond_jump_match.group(1) == 'jio' else register_value % 2 == 0

        if should_jump:
            has_jumped = True
            line_number += int(cond_jump_match.group(3))

    if not has_jumped:
        line_number += 1
    return line_number, a, b

line_number = 0
a = 0
b = 0
instruction = 0
while line_number >= 0 and line_number < len(input):
    line_number, a, b = read_instruction(line_number, a, b)
    instruction += 1
    if instruction % 10000 == 0:
        print(f'Step {instruction}, on line number {line_number}. a = {a}, b = {b}')

print(f'\nThe final value of register b is {b}')

a = 1
b = 0
instruction

In [ ]:
import itertools, math

packages = [int(line.strip()) for line in open('input24.txt', 'r').readlines()]

def divide_packages(packages, groups):
    group_weight = sum(packages) // groups
    minimum_number = 1
    while sum(packages[-minimum_number:]) < group_weight:
        minimum_number += 1
    
    possible_packages = itertools.combinations(packages, minimum_number)
    good_choices = [c for c in possible_packages if sum(c) == group_weight]
    while len(good_choices) == 0:
        minimum_number += 1
        possible_packages = itertools.combinations(packages, minimum_number)
        good_choices = [c for c in possible_packages if sum(c) == group_weight]
    return min(math.prod(choice) for choice in good_choices)

print(f'The best entanglement is {divide_packages(packages, 3)}')
print(f'The best entanglement with four trunks is {divide_packages(packages, 4)}')

In [ ]:
import re, math

input = open('input25.txt', 'r').read()
match = re.search('code at row ([0-9]+), column ([0-9]+)', input)
row = int(match.group(1))
column = int(match.group(2))

def get_number(row, column):
    n = column + row - 1
    return  n * (n + 1) // 2 - row + 1

code_number = get_number(row, column)
code = 20151125

def power_modulo(base, power, modulo):
    if power == 0:
        return 1
    if power == 1:
        return base
    running = base
    powers = [running]
    i = 1
    while 2**i <= power:
        running = (running ** 2) % modulo
        powers.append(running)
        i += 1
    multiplier = 1
    for i in range(len(powers)):
        if bin(power)[i+2] == '1':
            multiplier = (multiplier * powers[-i-1]) % modulo
    return multiplier

base = 252533
modulo = 33554393

print(f'The code at row {row} and column {column} is {(code * power_modulo(base, code_number - 1, modulo)) % modulo}')
    